# Wrappers

- We use Wrappers to imbue additional functions / processes to Agents

- Example of Wrappers:
    - PlanningWrappers: How to plan and execute the plan, self-correct plan if there are any errors
    - ReflectionWrappers: How to reflect upon what has been learned during task / across task and consolidate info for the next task
    - VerifierWrappers: How to verify agent's outputs and correct them accordingly
    - ConversationWrappers: How to converse with the agent
    - MultiAgentWrappers: How multiple agents can converse with one another

## Some Initial Imports

In [5]:
from taskgen import *

In [6]:
import os
# Important: Make sure you do not commit your own API key
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

In [7]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

## Define your Wrapper Here

In [17]:
class AgentWrapper(Agent):
    ''' This class takes an Agent imbues it with additional functions
    You can do everything the base Agent can do with this Wrapper '''
    
    def __init__(self, agent: Agent):
        # Initialize the parent Agent
        super().__init__(**agent.__dict__)  # Inherit all of the attributes of the passed agent
        # Initiatlize the functions
        self.assign_functions(list(agent.function_map.values()))
        
        ## Define Additional Variables as Needed
        self.additional_variable = True
       
    ### Additional Functions Here - Modify these yourself ###
    def answer(self, question, output_format = {'Answer': 'Concise Answer'}):
        ''' This is an example function to answer based on subtasks completed
        Note: This is just an example function, change it to whatever you want
        
        question (str): The question the user wants to ask
        output_format (dict): The output format in a dictionary'''
        return self.query(
f'''Answer the following question according to the subtasks completed: ```{question}```\n
Subtasks Completed: ```{self.subtasks_completed}```
Be concise and just give the answer with no explanation required.''', 
                               output_format = output_format)

## Show how the Wrapper is Used

In [18]:
agent = AgentWrapper(Agent('General Agent', 'Does Tasks', llm = llm))

In [19]:
agent.run('Find out 3+3')

Observation: No subtasks have been completed yet for the assigned task of finding out 3+3.
Thoughts: The task is straightforward and requires a simple calculation. Since the task is basic arithmetic, I can directly compute the result without needing to use a complex function.
Subtask identified: Calculate the sum of 3 and 3.
Getting LLM to perform the following task: Calculate the sum of 3 and 3.
> The assigned subtask was to calculate the sum of 3 and 3. By performing the addition operation, I combined the two numbers: 3 + 3. The result of this calculation is 6. Therefore, the detailed outcome of the task is that the sum of 3 and 3 equals 6.

Observation: The assigned subtask of calculating the sum of 3 and 3 has been completed, resulting in the final answer of 6.
Thoughts: Since the calculation is complete and the result is known, the next step is to finalize the task and provide the output to the user.
Subtask identified: End Task
Task completed successfully!



['The assigned subtask was to calculate the sum of 3 and 3. By performing the addition operation, I combined the two numbers: 3 + 3. The result of this calculation is 6. Therefore, the detailed outcome of the task is that the sum of 3 and 3 equals 6.']

In [20]:
# default answer
agent.answer('Give the final answer')

{'Answer': 6}

In [21]:
# customised answer in words
agent.answer('Give the final answer', {'Answer': 'in words, type: str'})

{'Answer': 'six'}

# Example: Conversation Wrapper

In [26]:
from termcolor import colored

class ConversationWrapper(Agent):
    ''' This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: None
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
    - ConversableAgent will automatically implement 3 new variables in `agent.shared_variables`:
        - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
        - **Conversation**: The entire history of the conversation
        - **Summary of Conversation**: A summary of the current conversation
        
    - ConversableAgent uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message'''
    
    def __init__(self, agent: Agent, persistent_memory: dict = None, person = 'User', conversation = None, num_past_conversation: int = 5, verbose: bool = True):
        # Initialize the parent Agent
        super().__init__(**agent.__dict__)  # Inherit all of the attributes of the passed agent
        # Initiatlize the functions
        self.assign_functions(list(agent.function_map.values()))
        
        ## Define Additional Variables as Needed
        self.persistent_memory = persistent_memory
        self.num_past_conversation = num_past_conversation
        self.person = person
        self.verbose = verbose
        
        ''' Define some external variables for the Agent '''
        # add in the various types of memory
        self.shared_variables['Persistent Memory'] = {}
        # add in the conversation
        if conversation is None:
            self.shared_variables['Conversation'] = []
        else:
            self.shared_variables['Conversation'] = conversation
        # add in the summary of conversation
        self.shared_variables['Summary of Conversation'] = ''
    
    ## Reply the person
    def chat(self, cur_msg):
        ''' This does one chat with the person, firstly performing actions then replying the person, while updating the important memory '''
        actions_done = []
        
        ## Do actions before replying person only if there are actions other than use_llm and end_task
        my_actions = list(self.function_map.keys()) 
        if 'use_llm' in my_actions: my_actions.remove('use_llm')
        if 'end_task' in my_actions: my_actions.remove('end_task')
        if len(my_actions) > 0:
            self.reset()
            self.run(f'''Summary of Past Conversation: ```{self.shared_variables['Summary of Conversation']}```
Past Conversation: ```{self.shared_variables['Conversation'][-self.num_past_conversation:]}```
Latest input from {self.person}: ```{cur_msg}```
Use Equipped Functions other than use_llm to help answer the latest input from {self.person}''',
            )
            
            if len(self.subtasks_completed) > 0:
                actions_done = self.reply_user('Summarise Subtasks Completed in one line', verbose = False)
                print(colored(f'Actions Done: {actions_done}', 'red', attrs = ['bold']))
                print()
                self.reset()

        ## Replies the person
        res = self.query(f'''Summary of Past Conversation: ```{self.shared_variables['Summary of Conversation']}```
Past Conversation: ```{self.shared_variables['Conversation'][-self.num_past_conversation:]}```
Latest Input from {self.person}: ```{cur_msg}```
Actions Done for Latest Input: ```{actions_done}```
Persistent Memory: ```{self.shared_variables['Persistent Memory']}```
Use Global Context and Conversation and Actions Done for Latest Input and and Persistent Memory as context when replying.

First think through how to reply the latest message by {self.person}, before drafting the reply.
{self.person} is not aware of Actions Done for Latest Input - include relevant information in your reply to {self.person}. Do not hallucinate actions.
Thereafter, update the Summary of Conversation''', 
                          
output_format = {"Thoughts": f"How to reply",
                 f"Reply to {self.person}": f"Your reply as {self.agent_name}",
                 "Summary of Conversation": "Summarise key points of entire conversation in at most two sentences, building on previous Summary"})
        
        # Update the Summary of Conversation and Append the conversation
        self.shared_variables['Summary of Conversation'] = res['Summary of Conversation']
        # Append information about user and actions to conversation
        self.shared_variables['Conversation'].append(f'{self.person}: {cur_msg}')
        self.shared_variables['Conversation'].append(f'{self.agent_name}: {res[f"Reply to {self.person}"]}')
        
        ## Update Persistent Memory
        if self.persistent_memory is not None and self.persistent_memory != {}:
            persistent_memory = strict_json(f'Update all fields of Persistent Memory based on information in Additional Conversation. Current value: ```{self.shared_variables["Persistent Memory"]}```',
               f'Additional Conversation\n{self.person}: {cur_msg}\n{self.agent_name}: {res[f"Reply to {self.person}"]}',
               output_format = self.persistent_memory,
               model = self.kwargs.get('model', 'gpt-4o-mini'),
               llm = self.llm,
               verbose = self.debug)
                                                           
            self.shared_variables["Persistent Memory"] = persistent_memory
        
        if self.verbose:
            print(colored(f'Thoughts: {res["Thoughts"]}', 'green', attrs = ['bold']))
            print(colored(f'Persistent Memory: {self.shared_variables["Persistent Memory"]}', 'blue', attrs = ['bold']))
            print(colored(f'Summary of Conversation: {res["Summary of Conversation"]}', 'magenta', attrs = ['bold']))
        
        return res[f'Reply to {self.person}']

In [33]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = llm)

# Define the ConversationWrapper
my_agent = ConversationWrapper(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
# while True:
#     user_input = input('User: ')
#     if user_input == 'exit': break
#     reply = my_agent.chat(user_input)
#     print(my_agent.agent_name + ':', reply)
#     print()

# Simulate an example conversation
user_input = 'I am happy'
reply = my_agent.chat(user_input)
print(my_agent.agent_name + ':', reply)
print()

user_input = 'I can develop TaskGen full-time now'
reply = my_agent.chat(user_input)
print(my_agent.agent_name + ':', reply)
print()

Thoughts: The user has expressed happiness, which is a positive emotion. It’s important to acknowledge this feeling and encourage the user to share more about what is making them happy.
Persistent Memory: {'User Requests for the Conversation': 'User shared their happiness and was asked to elaborate on its source.', 'User Emotion': 'happy', 'Summary of Key Incidents': 'User expressed happiness and was encouraged to discuss its causes.'}
Summary of Conversation: The user expressed happiness, and I encouraged them to share more about the source of their positive feelings.
Psychology counsellor: I’m glad to hear that you are feeling happy! Would you like to share what’s contributing to your happiness today?

Thoughts: The user is expressing excitement about being able to work on TaskGen full-time, which likely contributes to their happiness. I should acknowledge this achievement and encourage them to share more about their plans or feelings regarding this new opportunity.
Persistent Memory

In [34]:
# see the agent's conversation so far
my_agent.shared_variables['Conversation']

['User: I am happy',
 'Psychology counsellor: I’m glad to hear that you are feeling happy! Would you like to share what’s contributing to your happiness today?',
 'User: I can develop TaskGen full-time now',
 'Psychology counsellor: That’s fantastic news! Being able to develop TaskGen full-time sounds like a wonderful opportunity. What aspects of this project are you most excited about, and how do you feel about this new chapter in your work?']

In [35]:
my_agent.run('Write a professional summary of the conversation')

Observation: No subtasks have been completed yet for the assigned task of writing a professional summary of the conversation.
Thoughts: To complete the assigned task, I need to gather the key points and emotions expressed during the conversation to create a coherent summary.
Subtask identified: Utilize the equipped function to generate a general output that can help in summarizing the conversation based on the context provided.
Getting LLM to perform the following task: Utilize the equipped function to generate a general output that can help in summarizing the conversation based on the context provided.
> The conversation has revolved around understanding the User's emotional state and the context of their situation. The User has expressed feelings of confusion and anxiety regarding a recent life change. It is important to acknowledge these emotions and validate the User's experiences. A summary of the conversation highlights the User's concerns about their future and the impact of the

["The conversation has revolved around understanding the User's emotional state and the context of their situation. The User has expressed feelings of confusion and anxiety regarding a recent life change. It is important to acknowledge these emotions and validate the User's experiences. A summary of the conversation highlights the User's concerns about their future and the impact of their current situation on their mental well-being. Recommendations for coping strategies, such as mindfulness practices and seeking support from friends or professionals, have been discussed. Overall, the conversation aims to provide the User with clarity and reassurance, encouraging them to take proactive steps towards managing their emotions and navigating their circumstances."]